In [3]:
import pandas as pd
import numpy
from mygeotab import API, dates
import json
from pandas.io.json import json_normalize
import datetime 
import  time
from pandas import DataFrame as df
import scipy
from scipy.interpolate import interp1d

In [4]:
# pandas.options.display.max_rows
pd.options.display.max_rows = 200
pd.set_option('display.precision',7)
pd.set_option('display.max_columns', None)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Calcualting $\Delta$E using DiagnosticElectricVehicleBatteryPowerId logs
 - trips_10_3 - trips that occured on 10.3.2020.
 - status_data_10_3 - status data recorded on 10.3.2020.

In [5]:
trips_10_3 = pd.read_csv("trips_10_3.csv") 
status_data_10_3 = pd.read_csv("status_data_10_3.csv")

In [6]:
trips_10_3

,afterHoursDistance,afterHoursDrivingDuration,afterHoursEnd,afterHoursStart,afterHoursStopDuration,averageSpeed,distance,drivingDuration,idlingDuration,isSeatBeltOff,maximumSpeed,nextTripStart,speedRange1,speedRange1Duration,speedRange2,speedRange2Duration,speedRange3,speedRange3Duration,start,stop,stopDuration,workDistance,workDrivingDuration,workStopDuration,driver,id,stopPoint.x,stopPoint.y,device.id
0,12.4376540,00:18:04.9369999,True,True,00:01:11.1270000,41.270187,12.4376540,00:18:04.9369999,00:00:05,False,107.0,2020-03-10 06:48:00.127000+00:00,0,00:00:00,0,00:00:00,0,00:00:00,2020-03-10 06:28:44.063000+00:00,2020-03-10 06:46:49+00:00,00:01:11.1270000,0.0000000,00:00:00,00:00:00,UnknownDriverId,b3806,20.8907013,52.0766983,b4
1,49.8165900,00:50:12.8730000,True,True,04:21:46.9999999,59.524487,49.8165900,00:50:12.8730000,00:00:02,False,135.0,2020-03-10 14:31:20.062000+00:00,2,00:00:44.9480000,3,00:00:11.7210000,0,00:00:00,2020-03-10 06:48:00.127000+00:00,2020-03-10 07:38:13+00:00,06:53:07.0629999,0.0000000,00:00:00,02:31:20.0630000,UnknownDriverId,b3852,20.7886333,52.3743744,b4
2,0.0000000,00:00:00,False,False,00:00:00,22.287205,1.7833726,00:04:48.0640000,00:00:00,False,103.0,2020-03-10 14:36:24+00:00,0,00:00:00,0,00:00:00,0,00:00:00,2020-03-10 14:31:20.063000+00:00,2020-03-10 14:36:08.127000+00:00,00:00:15.8730000,1.7833726,00:04:48.0640000,00:00:15.8730000,UnknownDriverId,b385E,20.7851772,52.3736076,b4
3,0.0000000,00:00:00,False,False,00:00:00,15.962598,16.1712780,01:00:47.0630000,00:02:24,False,0.0,2020-03-10 16:00:53.063000+00:00,0,00:00:00,0,00:00:00,0,00:00:00,2020-03-10 14:36:24+00:00,2020-03-10 15:37:11.063000+00:00,00:23:42,16.1712780,01:00:47.0630000,00:23:42,UnknownDriverId,b3862,20.9611015,52.2751503,b4
4,0.0000000,00:00:00,False,False,00:00:00,23.541311,3.4527256,00:08:48,00:00:00,False,72.0,2020-03-10 16:41:48.063000+00:00,0,00:00:00,0,00:00:00,0,00:00:00,2020-03-10 16:00:53.063000+00:00,2020-03-10 16:09:41.063000+00:00,00:32:07,3.4527256,00:08:48,00:32:07,UnknownDriverId,b386C,20.9823227,52.2578163,b4
5,0.0000000,00:00:00,False,False,00:00:00,33.542732,27.8497850,00:49:49,00:00:00,False,90.0,2020-03-10 18:02:18.063000+00:00,0,00:00:00,0,00:00:00,0,00:00:00,2020-03-10 16:41:48.063000+00:00,2020-03-10 17:31:37.063000+00:00,00:30:41,27.8497850,00:49:49,00:30:41,UnknownDriverId,b38A4,20.8444672,52.2470894,b4
6,0.0000000,00:00:00,False,False,00:00:00,37.107970,13.7189920,00:22:10.9369999,00:14:06,False,144.0,2020-03-10 20:09:00+00:00,0,00:00:00,2,00:00:36.9040000,0,00:00:00,2020-03-10 18:02:18.063000+00:00,2020-03-10 18:24:29+00:00,01:44:31,13.7189920,00:22:10.9369999,01:44:31,UnknownDriverId,b38BE,20.9279480,52.2936058,b4
7,1.4832153,00:05:36,True,False,09:06:10.0630000,40.217567,37.9385720,00:56:36,00:15:01,False,144.0,2020-03-11 06:11:46.063000+00:00,0,00:00:00,4,00:02:13.0230000,0,00:00:00,2020-03-10 20:09:00+00:00,2020-03-10 21:05:36+00:00,09:06:10.0630000,36.4553570,00:51:00,00:00:00,UnknownDriverId,b3975,20.9609737,52.2748680,b4


In [7]:
energy_out_data = status_data_10_3.loc[status_data_10_3['diagnostic.id'] == 'DiagnosticElectricEnergyOutId'].copy()
energy_out_data.sort_values(by=['dateTime'], inplace=True)
energy_in_data = status_data_10_3.loc[status_data_10_3['diagnostic.id'] == 'DiagnosticElectricEnergyInId'].copy()
energy_in_data.sort_values(by=['dateTime'], inplace=True)

In [8]:
energy_out_data

,data,dateTime,controller,version,id,device.id,diagnostic.id
16,148.34,2020-03-10 06:33:07.477000+00:00,ControllerNoneId,0000000000058194,aPor5rVZpHXtSm-RbahDZBw,b4,DiagnosticElectricEnergyOutId
142,150.35,2020-03-10 06:40:22.117000+00:00,ControllerNoneId,0000000000058213,aiIa08IQQaoAwu3mKOEJyBA,b4,DiagnosticElectricEnergyOutId
177,150.53,2020-03-10 06:43:27.090000+00:00,ControllerNoneId,00000000000582d6,aH5wsNsWP9LzuPmB0QD2UAA,b4,DiagnosticElectricEnergyOutId
410,151.07,2020-03-10 06:55:11.220000+00:00,ControllerNoneId,000000000005831f,aH5yRtXk7UxeDWcya2SURBA,b4,DiagnosticElectricEnergyOutId
566,153.08,2020-03-10 07:06:26.167000+00:00,ControllerNoneId,00000000000583bb,aiXLkAMFhY7IQn0z_VN6AJA,b4,DiagnosticElectricEnergyOutId
878,159.09,2020-03-10 07:28:48.113000+00:00,ControllerNoneId,00000000000584f2,aq9dZ-CSsgY3LN5G3QfzhBw,b4,DiagnosticElectricEnergyOutId
1057,161.46,2020-03-10 07:38:15.127000+00:00,ControllerNoneId,00000000000585a6,avUSH1CbKwzfuyvM6kIdYCQ,b4,DiagnosticElectricEnergyOutId
1223,161.46,2020-03-10 14:36:06.147000+00:00,ControllerNoneId,00000000000587d1,a72iVYiU40m_pT5uKFaG9Aw,b4,DiagnosticElectricEnergyOutId
1417,165.15,2020-03-10 15:58:22.933000+00:00,ControllerNoneId,0000000000058894,aoYO-x_dOVr-9bH2gx1GBEA,b4,DiagnosticElectricEnergyOutId
1426,165.16,2020-03-10 15:59:02.823000+00:00,ControllerNoneId,000000000005889d,anVgQQmumNaYsSBxv2VMDBQ,b4,DiagnosticElectricEnergyOutId


In [9]:
energy_in_data

,data,dateTime,controller,version,id,device.id,diagnostic.id
14,25.56,2020-03-10 06:33:07.477000+00:00,ControllerNoneId,0000000000058192,aPor5rVZpHXtSm-RbahDZBQ,b4,DiagnosticElectricEnergyInId
382,26.19,2020-03-10 06:46:54.133000+00:00,ControllerNoneId,0000000000058303,aH5wsNsWP9LzuPmB0QD2ULQ,b4,DiagnosticElectricEnergyInId
408,26.19,2020-03-10 06:55:11.220000+00:00,ControllerNoneId,000000000005831d,aH5yRtXk7UxeDWcya2SURAg,b4,DiagnosticElectricEnergyInId
956,28.20,2020-03-10 07:33:52.090000+00:00,ControllerNoneId,0000000000058540,aefAA2hloXPvYvRhLRH_hBg,b4,DiagnosticElectricEnergyInId
1055,28.48,2020-03-10 07:38:15.127000+00:00,ControllerNoneId,00000000000585a4,avUSH1CbKwzfuyvM6kIdYBw,b4,DiagnosticElectricEnergyInId
1221,28.48,2020-03-10 14:36:06.147000+00:00,ControllerNoneId,00000000000587cf,a72iVYiU40m_pT5uKFaG9AQ,b4,DiagnosticElectricEnergyInId
1415,29.45,2020-03-10 15:58:22.933000+00:00,ControllerNoneId,0000000000058892,aoYO-x_dOVr-9bH2gx1GBDg,b4,DiagnosticElectricEnergyInId
1424,29.45,2020-03-10 15:59:02.823000+00:00,ControllerNoneId,000000000005889b,anVgQQmumNaYsSBxv2VMDAw,b4,DiagnosticElectricEnergyInId
1442,29.45,2020-03-10 15:59:20.797000+00:00,ControllerNoneId,00000000000588ad,anVgQQmumNaYsSBxv2VMDFQ,b4,DiagnosticElectricEnergyInId
1691,29.87,2020-03-10 16:09:41.653000+00:00,ControllerNoneId,00000000000589a5,abtGgZf-VLsDkutcPFAXSDw,b4,DiagnosticElectricEnergyInId


###  1.st example
Calculating $\Delta$E in between two logs(TotalEnergyIn, TotalEnergyOut)
- energyIn and EnergyOut closest to the start and end of the trip, for first trip that occured on 10.3.2020.(2020-03-10 06:28:44.063000+00:00 - 2020-03-10 06:46:49+00:00) is taken and compared with the result achieved by calculating integral of power in the same time period

In [10]:
energy_in_first_trip = energy_in_data['data'].values[1] - energy_in_data['data'].values[0]
energy_out_first_trip = energy_out_data['data'].values[2] - energy_out_data['data'].values[0]
energy_first_trip = energy_out_first_trip - energy_in_first_trip

print('Correct energy in during the first trip:              ' + str(energy_in_first_trip))
print('Correct energy out during the first trip:             ' + str(energy_out_first_trip))
print('Correct total amount of energy during the first trip: ' + str(energy_first_trip))

Correct energy in during the first trip:              0.6300000000000026
Correct energy out during the first trip:             2.1899999999999977
Correct total amount of energy during the first trip: 1.5599999999999952


In [11]:
status_data_first_trip = status_data_10_3.loc[(status_data_10_3['dateTime'] > trips_10_3['start'][0]) & (status_data_10_3['dateTime'] < trips_10_3['stop'][0])]
battery_power_data = status_data_first_trip.loc[status_data_first_trip['diagnostic.id'] == 'DiagnosticElectricVehicleBatteryPowerId'].copy()
battery_power_data.sort_values(by=['dateTime'], inplace=True)
data = battery_power_data['data'].tolist()
times = battery_power_data['dateTime'].tolist()

x = numpy.asarray([time.mktime(datetime.datetime.strptime(t.split('+')[0], "%Y-%m-%d %H:%M:%S.%f").timetuple()) for t in times]) 
x = x.astype('float64')
y = numpy.asarray(data)
y = y.astype('float64') 

finterp = scipy.interpolate.InterpolatedUnivariateSpline(x, y, k=1)
en_in = 0
en_out = 0
total = 0
for i in range(len(x)-1):
    fract = finterp.integral(x[i], x[i+1])
    if fract<0:
        en_in -= fract
    else:
        en_out += fract
    total += fract
    
print('Calculated energy in during the first trip:    ' + str(en_in / (1000 * 3600)))
print('Calculated energy out during the first trip:   ' + str(en_out / (1000 * 3600)))
print('Calculated total energy during the first trip: ' + str(total / (1000 * 3600)))

Calculated energy in during the first trip:    0.3754823611111111
Calculated energy out during the first trip:   2.6221294444444445
Calculated total energy during the first trip: 2.2466470833333334


### 2.nd example
Calculating $\Delta$E for all the trips of the day, comparing final result with total daily consumption calculated from TotalEnergyIn and TotalEnergyOut logs.

In [12]:
correct_total_en_in_10_3 = energy_in_data['data'].values[-1] - energy_in_data['data'].values[0]
correct_total_en_out_10_3 = energy_out_data['data'].values[-1] - energy_out_data['data'].values[0]
correct_total_en_10_3 = correct_total_en_out_10_3 - correct_total_en_in_10_3

print('Corect total energy in : ' + str(correct_total_en_in_10_3))
print('Corect total energy out: ' + str(correct_total_en_out_10_3))
print('Corect total energy    : ' + str(correct_total_en_10_3))

Corect total energy in : 9.080000000000002
Corect total energy out: 37.66
Corect total energy    : 28.579999999999995


In [13]:
calculated_total_in_10_3 = 0
calculated_total_out_10_3 = 0
calculated_total_10_3 = 0

for index,trip in trips_10_3.iterrows():
    
    status_data = status_data_10_3.loc[(status_data_10_3['dateTime'] > trip['start']) & (status_data_10_3['dateTime'] < trip['stop'])]
    battery_power_data = status_data.loc[status_data['diagnostic.id'] == 'DiagnosticElectricVehicleBatteryPowerId'].copy()
    battery_power_data.sort_values(by=['dateTime'], inplace=True)
    
    data = battery_power_data['data'].tolist()
    times = battery_power_data['dateTime'].tolist()

    x = numpy.asarray([time.mktime(datetime.datetime.strptime(t.split('+')[0], "%Y-%m-%d %H:%M:%S.%f").timetuple()) for t in times])   
    x = x.astype('float64')
    y = numpy.asarray(data)
    y = y.astype('float64') 
    
    if len(x)<2 or len(y)<2:
        continue
    
    finterp = scipy.interpolate.InterpolatedUnivariateSpline(x, y, k=1)
    
    for i in range(len(x)-1):
        fract = finterp.integral(x[i], x[i+1])
        if fract < 0 :
            calculated_total_in_10_3 -= fract
        else:
            calculated_total_out_10_3 += fract
        calculated_total_10_3 += fract
        
print('Calculated total energy in : '+str(calculated_total_in_10_3/ (1000 * 3600)))
print('Calculated total energy out: '+str(calculated_total_out_10_3/ (1000 * 3600)))
print('Calculated total energy    : '+str(calculated_total_10_3/ (1000 * 3600)))

Calculated total energy in : 5.2611475
Calculated total energy out: 28.000515
Calculated total energy    : 22.7393675
